In [ ]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import time
import random
import os

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_from_url(fn, url):
    if(os.path.exists(fn)):
        print(fn)
        return open(fn, 'rb').read()
    
    time.sleep(random.randint(2, 5))
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}
    responce = requests.get(url, verify = False, headers = header)
    print(url, responce.status_code)

    if responce.status_code == 200:
        if fn != '':
            file = open(fn, 'wb')
            file.write(responce.content)
            file.close()
        return responce.text
    else:
        return ''

In [ ]:
def parse_list(content):
    res = []
    soup = BeautifulSoup(content, 'lxml')
    table = soup.body.find('table', {"class": "custom_table"})
    for row in table.tbody.childGenerator():
        if(row.name=='tr'):
            url = row.find('a')
            print(url['href'])
            res.append(url['href'])
    return res

In [ ]:
def parse_case(content):
    res = []
    soup = BeautifulSoup(content, 'lxml')
    table = soup.body.find('div', {'class', 'cardsud_wrapper'})
    if table.find('a') != None:
        url = table.find('a')['href']
        return 'https://www.mos-gorsud.ru' + url
    return ''

In [ ]:
def parse_info(content):
    soup = BeautifulSoup(content, 'lxml')
    table = soup.body.find('table', {'class', 'custom_table mainTable'})
    for row in table.tbody.childGenerator():
        if row.name == 'tr':
            key = cut_word(row.find_all('td')[1].text)
            if(key == 'Мотивированноерешение'):
                if row.find('a') != None:
                    url = row.find('a')['href']
                    if url != '#':
                        return 'https://www.mos-gorsud.ru' + url
    return ''

In [ ]:
def cut_word(text):
    return text.replace(' ','').replace('\n', '')

In [ ]:
#for page in range(1000, 1030):
def get_page(page):
    n = 0
    content = get_from_url('', 'https://mos-gorsud.ru/mgs/search?caseDateFrom=&caseDateTo=01.08.2022&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&caseNumber=&letterNumber=&category=&codex=&courtAlias=mgs&docsDateFrom=&docsDateTo=&documentStatus=&documentText=&documentType=&instance=2&judge=&participant=&processType=2&publishingState=&uid=&year=&formType=fullForm&page='+str(page))
    cases = parse_list(content)
    for case in cases:
        n += 1
        name = str(page).zfill(5)+str(n).zfill(2)
        content = get_from_url('case/' + name + '.htm', 'https://mos-gorsud.ru'+case)
        url = parse_case(content)
        if url != '':
            info = get_from_url('info/' + name + '.htm', url)
            url = parse_info(info)
            if url != '':
                get_from_url('doc/' + name + '.doc', url)

In [ ]:
n1 = 8269
n2 = 8300
page = n1
while page<n2:
    try:
        get_page(page)
    except:
        print("exception", page)
    else:
        page +=1